<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Creating-a-model" data-toc-modified-id="Creating-a-model-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Creating a model</a></span></li><li><span><a href="#Feature-Importance" data-toc-modified-id="Feature-Importance-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Feature Importance</a></span><ul class="toc-item"><li><span><a href="#Mean-Decrease-Impurity" data-toc-modified-id="Mean-Decrease-Impurity-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Mean Decrease Impurity</a></span></li><li><span><a href="#Permutation-Importance" data-toc-modified-id="Permutation-Importance-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Permutation Importance</a></span></li></ul></li><li><span><a href="#Feature-Contributions" data-toc-modified-id="Feature-Contributions-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Feature Contributions</a></span><ul class="toc-item"><li><span><a href="#Plotting-feature-contributions" data-toc-modified-id="Plotting-feature-contributions-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Plotting feature contributions</a></span><ul class="toc-item"><li><span><a href="#Boxplots" data-toc-modified-id="Boxplots-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Boxplots</a></span></li><li><span><a href="#Swarmplots" data-toc-modified-id="Swarmplots-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>Swarmplots</a></span></li><li><span><a href="#Plotting-Feature-Contributions-against-Feature-Values" data-toc-modified-id="Plotting-Feature-Contributions-against-Feature-Values-3.1.3"><span class="toc-item-num">3.1.3&nbsp;&nbsp;</span>Plotting Feature Contributions against Feature Values</a></span></li><li><span><a href="#Heatmaps" data-toc-modified-id="Heatmaps-3.1.4"><span class="toc-item-num">3.1.4&nbsp;&nbsp;</span>Heatmaps</a></span></li></ul></li></ul></li><li><span><a href="#Joint-Feature-Contributions" data-toc-modified-id="Joint-Feature-Contributions-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Joint Feature Contributions</a></span></li><li><span><a href="#PDP-and-ICE-plots" data-toc-modified-id="PDP-and-ICE-plots-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>PDP and ICE plots</a></span><ul class="toc-item"><li><span><a href="#Centered-ICE-Plots" data-toc-modified-id="Centered-ICE-Plots-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Centered ICE Plots</a></span></li><li><span><a href="#Two-dimensional-PDPs" data-toc-modified-id="Two-dimensional-PDPs-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Two-dimensional PDPs</a></span></li></ul></li><li><span><a href="#LIME" data-toc-modified-id="LIME-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>LIME</a></span></li><li><span><a href="#SHAP" data-toc-modified-id="SHAP-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>SHAP</a></span></li><li><span><a href="#Resources" data-toc-modified-id="Resources-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Resources</a></span></li></ul></div>

Интерпретация моделей машиннго обучения на Python


В этой статье мы исследуем различные инструменты и методы, которые можно использовать для интерпретации моделей «черного ящика».

Поскольку сегодня проходит <a href='https://ru.wikipedia.org/wiki/%D0%94%D1%80%D0%B0%D1%84%D1%82_%D0%9D%D0%A4%D0%9B'>драфт НФЛ</a>, я думаю, что было бы забавным использовать данные по <a href='https://en.wikipedia.org/wiki/NFL_Scouting_Combine'>НФЛ-комбинату</a>(ежегодных соревнований с целью выявить перспективных игроков). Используя эти данные, мы сначала построим модель для прогнозирования результатов начала карьеры играков защиты - <a href='https://ru.wikipedia.org/wiki/%D0%94%D0%B5%D1%84%D0%B5%D0%BD%D1%81%D0%B8%D0%B2_%D1%8D%D0%BD%D0%B4'>Дефенсив энд (англ. Defensive end, D-End) (DE)</a> . После этого мы будем использовать различные методы интерпретации для лучшего понимания модели.

# Создание модели

Давайте начнем с импорта того, что нам нужно для настройки данных и нашего конвейера моделирования.

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

from sklearn.preprocessing import Imputer
from sklearn.model_selection import  cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestRegressor

from skll.metrics import spearman

from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

import warnings

In [2]:
# устанавлиаем настройки отображения графиков
sns.set(style="white", palette="colorblind", font_scale=1.2, 
        rc={"figure.figsize":(12,9)})
RANDOM_STATE = 420
N_JOBS=8

Здесь мы загружаем данные, которые содержат общую информацию о всех игроках и их приблизительную стоимость (https://www.pro-football-reference.com/blog/index37a8.html) за 3 года, которую мы будем использовать в качестве показателя ранних результатов карьеры (и в дальнейшем мы будем сокращенно обозначать как <a href='https://www.pro-football-reference.com/blog/index37a8.html'>AV</a>).

При построении нашей модели мы не будем предсказывать AV игроков защиты, вместо этого мы будем предсказывать его AV-ранг (в процентах) среди других защитников. Игроки с высоким AV должны быть ближе к 1, а игроки с низким AV - ближе к 0. 

**ПРИМЕЧАНИЕ:** Если вы хотите узнать, какие измеримые[combine measurables] [значения?имеют значение] для каждой позиции в НФЛ вы найдете ответы в [серии](http://harvardsportsanalysis.org/2015/02/the-nfl-combine-actually-matters/) посвященных этой теме [постов](http://harvardsportsanalysis.org/2015/02/the-combine-actually-matters-part-3-predicting-the-draft/) [Била Лоттера](https://twitter.com/bill_lotter?lang=en) в его [блоге](http://harvardsportsanalysis.org/2015/02/the-combine-actually-matters-part-2/).

In [3]:
# считываем данные
data_df = pd.read_csv('data/combine_data_since_2000_PROCESSED_2018-04-26.csv')
# выбираем игроков, которые состоят в лиге на протяжении трех лет или более
data_df2 = data_df.loc[data_df.Year <= 2015].copy()
# считаем AV[процент|перцентиль] игрока по его позиции [EN] calculate the player AV percentiles by position
data_df2['AV_pctile'] = (data_df2.groupby('Pos')
                                  .AV
                                  .rank(pct=True,
                                        method='min', 
                                        ascending=True))

FileNotFoundError: File b'data/combine_data_since_2000_PROCESSED_2018-04-26.csv' does not exist

Итак, мы загрузили данные и рассчитали [AV percentiles], приступим к отбору данных по защитникам (DE) и создадим обучающую и тестовую выборку. Мы обучим нашу модель на первых 8 годах (2000-2011) [?как-то не похоже на 8 лет?] и затем протестируем её на следующих четырех годах (2012-2015). 

In [ ]:
# Отберем данные по интересующим нас позициям, в данно случае это защитники - DE
pos_df = data_df2.loc[data_df2.Pos=='DE'].copy().reset_index(drop=True)

# Разделим данные на обучающую и тестовую выборку
train_df = pos_df.loc[pos_df.Year <= 2011]
test_df = pos_df.loc[pos_df.Year.isin([2012, 2013, 2014, 2015])]

We'll be build a random forest model since that seems to be the [most common](https://www.kaggle.com/surveys/2017) "black box" algorithm people use at work. One thing to to note is that in our modeling `Pipeline` we will need to include an `Imputer` because some DEs are missing data as they did not participate in all the combine drills.

Мы построим модель случайного леса, так как это [наиболее распространенный](https://www.kaggle.com/surveys/2017) алгоритм "черного ящика", который люди используют на практике. Следует отметить, что в наш `Pipeline` обработки данных мы должны будем включить `Imputer`, так как по части DE есть пропуски в данных, потому что они принимали участие не во всех соревнованиях.

In [ ]:
# Измеренные параметры
features = ['Forty',
            'Wt',
            'Ht',
            'Vertical',
            'BenchReps',
            'BroadJump',
            'Cone',
            'Shuttle']
# хто мы хотим предсказывать
target = 'AV_pctile'

X = train_df[features].values
y = train_df[target].values

# pipeline обработки данных
pipe = Pipeline([("imputer", Imputer()),
                 ("estimator", RandomForestRegressor(random_state=RANDOM_STATE))])

Для настройки нашей модели мы будем использовать `BayesSearchCV` из `scikit-optimize`, который использует байесовскую оптимизацию для поиска наилучших гиперпараметров. Также мы будем использовать коэффициент корреляции Спирмена как нашу метрику оценки (scoring metric),  так как нас в основном интересует рейтинг игроков, когда речь идет о драфте.

In [ ]:
# Мы используем коэффициент корреляции Спирмена как нашу метрику оценки
# и нас интересует ранг спортсменов
spearman_scorer = make_scorer(spearman)

# поиск гиперпараметров для различных стратегий импутации данных
rf_param_space = {
    'imputer__strategy': Categorical(['mean', 'median', 'most_frequent']),
    'estimator__max_features': Integer(1, 8),
    'estimator__n_estimators': Integer(50, 500), 
    'estimator__min_samples_split': Integer(2, 200),
}
# создаем объект поиска
search = BayesSearchCV(pipe, 
                      rf_param_space, 
                      cv=10,
                      n_jobs=N_JOBS, 
                      verbose=0, 
                      error_score=-9999, 
                      scoring=spearman_scorer, 
                      random_state=RANDOM_STATE,
                      return_train_score=True, 
                      n_iter=75)
# обучение модели
# я получил несколько странных предупреждений (funky warnings), возможно что из-за использования коэффициента корреляции Спирмена,
# и я предпочет поавить их вывод
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    search.fit(X, y) 

In [ ]:
# лучшие параметры модели
search.best_params_

In [ ]:
# CV score
search.best_score_

In [ ]:
# CV стандартное отклонение
search.cv_results_['std_test_score'][search.best_index_]

Теперь, когда мы настроили нашу модель, давайте оценим ее на тестовом наборе данных.

In [ ]:
# тестовый набор
X_test = test_df[features].values
y_test = test_df[target].values
# предсказание
y_pred = search.predict(X_test)
# оценка
model_test_score = spearman_scorer(search, X_test, y_test)
model_test_score



Таким образом, предсказания нашей модели имеют ранговую корреляцию Спирмена около 0,209. Хорошо это или плохо? Я не знаю. Чтобы лучше понять, насколько хороший или плохой результат, мы можем использовать фактические <a href='https://ru.wikipedia.org/wiki/%D0%94%D1%80%D0%B0%D1%84%D1%82_(%D1%81%D0%BF%D0%BE%D1%80%D1%82)'>пики драфта</a> в качестве способа сравнения.

In [ ]:
# создаем percentiles для пики драфта НФЛ
# Меньшее численное значение пики (e.g. 1, 2, 3) ранжированно ближе к 1
# Большее численное значение пики (e.g. 180, 200, etc) ранжированно ближе к 0
draft_pick_pctile = test_df.Pick.rank(pct=True,
                                      method='min', 
                                      ascending=False, 
                                      na_option='top')
spearman(y_test, draft_pick_pctile)

Похоже, что команды НФЛ в 3 раза лучше ранжируют DEs чем наша модель.

Теперь, когда у нас есть модель, давайте рассмотрим различные инструменты для её лучшего понимания.

# Значимость признаков

## Mean Decrease Impurity  ( усредненное уменьшение Джини ?)

Если вы испальзуете ансамбль деревьев решений, например случайный лес, вы можете оценить то, какие параметры модель считает более значимыми проверяя значимость признаков.  В `scikit-learn` значимость признаков отражает то, как признак уменьшает некий критерий. В нашей регрессионной моели этот критерий - средняя квадратичная ошибка.  Этот метод рассчета значимости признака обычно называется mean decrease impurity или gini importance.

Мы можем получить доступ к значимостям признаков нашей модели при помощи `feature_importances_` аттрибута.

In [ ]:
# берем estimator и imputer из нашего pipeline, 
# который будет использоваться, когда мы пытаемся интерпретировать нашу модель
estimator = search.best_estimator_.named_steps['estimator']
imputer = search.best_estimator_.named_steps['imputer']

estimator.feature_importances_

В качестве альтернативы, мы можем использовать `explain_weights_df` функцию  из пакета`eli5`, которая возвращает значимости и имена признаков, которые мы передадим в неё как `DataFrame` в `pandas`.

In [ ]:
import eli5

# создаем датафрейм значимости признаков
feat_imp_df = eli5.explain_weights_df(estimator, feature_names=features)
feat_imp_df

Похоже на то, что вес игрока и его результат в беге на 40 ярдов(<a href='https://ru.wikipedia.org/wiki/%D0%91%D0%B5%D0%B3_%D0%BD%D0%B0_40_%D1%8F%D1%80%D0%B4%D0%BE%D0%B2'>forty time</a>) являются значимыми признаками для модели. Нужно отметить, что `explain_weights_df` также возвращает стандартные отклонения, но они могут быть ненадежными, поскольку эти значения предполагают нормальное распределение. Вместо того, чтобы полагаться на эти стандартные отклонения, мы можем получить доступ к каждому дереву в нашем ансамбле и построить полное распределение значений функций.


In [ ]:
# получаем значимость признаков для каждого дерева и затем визуализируем их распределения как boxplots
all_feat_imp_df = pd.DataFrame(data=[tree.feature_importances_ for tree in 
                                     estimator],
                               columns=features)

(sns.boxplot(data=all_feat_imp_df)
        .set(title='Распределения значимости признаков',
             ylabel='Значимость'));

## Пермутированная  важность(?англ. Permutation Importance)

Важность перестановок(?англ. Permutation Importance) или среднее снижение точности(?англ. mean decrease accuracy (MDA)) это альтернатива примеси (критерий) Джини(?англ. mean decrease impurity) метрика, которая может быть использована в любой модели. Главная идея важности перестановок (?англ. permutation importance) это перестановка значений каждого предиктора и оценка того, насколько сильно эта перестановка негативно влияет на метрику скоринга (в нашем случае это клэфициент корреляции ранга Спирмена).  Это дает нам представление о том, как наша модель бует работать беэ этого конкретного признака. Все что нам нужно, это рассчитать важность перестановок используя `PermutationImportance` из `eli5`.

In [ ]:
from eli5.sklearn import PermutationImportance

# нам нужно импутировать анные преже, чем рассчитывать permutation importance
train_X_imp = imputer.transform(X)
# настраиваем met-estimator(?метапараметр?) для рассчета permutation importance на нашей учебной выборке данных
perm_train = PermutationImportance(estimator, scoring=spearman_scorer,
                                   n_iter=50, random_state=RANDOM_STATE)
# обучаем и смотрим permuation importances
perm_train.fit(train_X_imp, y)
eli5.explain_weights_df(perm_train, feature_names=features)

In [ ]:
# построим график распреелений
perm_train_feat_imp_df = pd.DataFrame(data=perm_train.results_,
                                      columns=features)
(sns.boxplot(data=perm_train_feat_imp_df)
        .set(title='Permutation Importance Распрееление (учебная выборка)',
             ylabel='Значимость'));

Основываясь на permutation importances мы снова видим что вес(Wt) и бег на 40 ярдов(Forty) это ва наших главных предиктора модели.

# Вклады признаков (англ. Feature Contributions)

Метрика значимости предикторов может подсказать нам идеи о том, какие параметры модели наиболее значимы, но она ничего не скажет нам о том, как эти параметры влияют на предсказания нашей модели. Единственный способ сделать это - использовать пути в наших деревьях чтобы увидеть насколько сильно каждый параметр меняет предсказание когда мы движемся от родительского узла к дочернему узлу.
В конце концов, мы можем разбить эти вклады параметров линейным образом, чтобы наши прогнозы можно было интерпретировать следующим образом: 

$$prediction = bias + feature_{1}contribution + feature_{2}contribution +... feature_{n}contribution$$


Андо Саабас написал [несколько](http://blog.datadive.net/interpreting-random-forests/) [статей](http://blog.datadive.net/random-forest-interpretation-conditional-feature-contributions/) [в блоге](http://blog.datadive.net/random-forest-interpretation-with-scikit-learn/)  для более подробного разбора этой темы. Сейчас я попытаюсь объяснить, как рассчитываются вклады параметров, рассмотрев пример, в котором в нашем лесу используется дерево небольшой глубины(англ. shallow tree). 

In [ ]:
from IPython.display import Image  
from sklearn.tree import export_graphviz
import graphviz
import pydotplus
from io import StringIO  

# код примеров взят из:
# https://medium.com/@rnbrown/creating-and-visualizing-decision-trees-with-python-f8e8fa394176
# Берем все деревья глубиною 2 в случайном лесу
depths2 = [tree for tree in estimator.estimators_ if tree.tree_.max_depth==2]
# выбираем первое
tree = depths2[0]
# строим график дерева
dot_data = StringIO()
export_graphviz(tree, out_file=dot_data, feature_names=features, 
                filled=True, rounded=True, special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

Чтобы упростить задачу, давайте оценим вклады параметров для игрока, который следует по левому пути в дереве выше, допустим, игрок, который весит 260 фунтов и пробегает сорок ярдов за 4,6 секунды.

Начинаем с корневого (верхнего) узла, где средний AV-процентиль всех сэмплов равен 0,414 (это значение нашего смещения). При первом разделении дерево учитывает, весит ли игрок 270,5 фунтов или меньше, поэтому любое изменение в прогнозе, вызванное этим разделением, связано с весом игрока. Наш игрок попадает в левый дочерний узел, потому что он весит 260 фунтов.

Since the average percentile at this node is 0.317 versus 0.414 in the parent node, we can say that the player's weight caused a decrease of 0.097 percentage points in his predicted AV percentile.  Now for the split at this new node, the tree considers whether the player runs a 4.755 forty or less. The player in our example runs a 4.6 forty, so after this split, he ends up in the leftmost leaf node of the tree. 

Поскольку средний процентиль в этом узле равен 0,317 против 0,414 в родительском узле, мы можем сказать, что вес игрока вызвал уменьшение его прогнозируемого AV-процентиля на 0,097 процентных пункта. Теперь для разделения на этом новом узле дерево учитывает, пробежал ли игрок сорок ярдов за 4,755 сек. или меньше. Игрок в нашем примере пробежал за 4.6 сек., поэтому после этого разделения он оказывается в крайнем левом листе дерева.

At this leaf node the player's final predicted percentile is 0.481, since that is an increase of 0.164 percentage points from the previous node, we can say that the player's forty time contributed 0.164 percentage points to his predicted AV percentile.
In the end, the feature contributions for this player's predicted AV percentile are as follows:

В этом листовом узле конечный прогнозируемый процентиль игрока равен 0,481, то есть он увеличился на 0,164 процентных пункта по сравнению с предыдущим узлом, и мы можем сказать, что результат игрока в беге на сорок ярдов добавил 0,164 процентных пункта в его прогнозируемый AV-процентиль.
В итоге, вклад в функцию для прогнозируемого AV-процентиля этого игрока выглядит следующим образом:

$$\underset{\text{AV %ile}}{0.481} = \underset{\text{bias}}{0.414}-\underset{\text{Wt}}{0.097}+\underset{\text{Forty}}{0.164}$$

Давайте посмотрим, какие значения вклаов параметров мы получим с помощью функции `explain_prediction_df` из библиотеки `eli5`.

In [ ]:
# простой пример игрока с результатом в беге 4.6 сек. и весом в 260 фунтов
example = np.array([4.6, 260, 0, 0, 0, 0, 0, 0])
eli5.explain_prediction_df(tree, example, feature_names=features)

Итак, мы можем ожидать прогноза нашего дерева в 0.481.

In [ ]:
tree.predict(example.reshape(1,-1))

Так и есть. Чтобы вычислить ? (aнгл. bias term) и вклады признаков для всего леса деревьев, все, что вам нужно сделать, это усреднить условия смещения(aнгл. bias terms) и вклады параметров для всех деревьев.

**NOTE:** Я знаю две библиотеки, которые позволяют легко вычислять эти виды вкладов параметров, `eli5` и  `treeintepretter`. Они отличаются друг от друга как API, так и алгоритмами, которые они могут использовать. `treeinterpretter` на данный момент работает для следующих эстиматоров `scikit-learn`:

- DecisionTreeRegressor
- DecisionTreeClassifier
- ExtraTreeRegressor
- ExtraTreeClassifier
- RandomForestRegressor
- RandomForestClassifier
- ExtraTreesRegressor
- ExtraTreesClassifier

`eli5` работает как для тех же, вдобавок также работает и для эстиматора градиентного бустинга в `scikit-learn` и  LightGBM-эстиматора.


Теперь давайте получим вклады параметров для каждой выборки в наших учебном и тестовом наборах данных. Помните, `explain_prediction_df` считает вклады наблюдений по одному, и может занимать много времени. Чтобы ускорить процесс, я написал вспомогательную функцию, которая позволяет нам использовать несколько процессов (то есть несколько ядер ЦП).

In [ ]:
from concurrent.futures import ProcessPoolExecutor

def multiproc_iter_func(max_workers, an_iter, func, item_kwarg, **kwargs):
    """
    Вспомогательная функция, которая многопоточно применяется к каждому элементу в итерируемой
    последовательности.
    'item_kwarg' это keyword аргумент элемента в итерируемой последовательности, которая передена в функцию
    """
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        future_results = [executor.submit(func, **{item_kwarg: item}, **kwargs)
                          for item in an_iter]

        results = [future.result() for future in future_results]
        
    return results

In [ ]:
# Создание учебного набора вкладов
# создание списка всех атрибутов обучающего набора данных
train_expl_list = multiproc_iter_func(N_JOBS, train_X_imp, 
                                      eli5.explain_prediction_df, 'doc',
                                      estimator=estimator, 
                                      feature_names=features)
# Конкатенируем их в 1 большой датафрейм, с правильным именем играка в качестве индекса
train_expl_df = pd.concat(train_expl_list, keys=train_df.Player, 
                          names=['Player'])
# Посмотрим на несколько первых игроков.
train_expl_df.head(18)

In [ ]:
# Создание тестового набора вкладов
# нам необходимо импутировать пропущенные значения в тестовом наборе
test_X_imp = imputer.transform(X_test)
# повторяем то, что мы сделали для обучающего набора данных
test_expl_list = multiproc_iter_func(N_JOBS, test_X_imp, 
                                     eli5.explain_prediction_df, 'doc', 
                                     estimator=estimator,
                                     feature_names=features)

test_expl_df = pd.concat(test_expl_list, keys=test_df.Player, 
                         names=['Player'])
test_expl_df.head(18)

In [ ]:
# Двойная проверка что суммы вкладов равны действительным предсказаниям
y_pred_sums = test_expl_df.groupby('Player').weight.sum()
np.allclose(y_pred, y_pred_sums)

## Построение графика вкладов признаков
Теперь, когда у нас есть все вклады признаков, построим несколько графиков, чтобы лучше разобраться в них.

### Ящики с усами(боксплот)
Для начала построим ящик с усами, чтобы увидеть чтобы увидеть распределения вкладов каждого признака.

In [ ]:
# Создадим большой датасет, который будет включать и учебный, и проверочный набор дынных.
# чтобы построить их график, используя боксплот от seaborn
train_expl_df.rename(columns={'weight': 'contribution'}, inplace=True)
test_expl_df.rename(columns={'weight': 'contribution'}, inplace=True)
train_expl_df['data'] = 'train'
test_expl_df['data'] = 'test'
train_test_expl_df = pd.concat([train_expl_df, test_expl_df])
sns.boxplot(x='feature', y='contribution', hue='data', order=features,
            data=train_test_expl_df.loc[train_test_expl_df.feature!='<BIAS>'],
            palette={'train': 'salmon', 
                     'test':'deepskyblue'})
plt.legend(loc=9)
plt.title('Распределение вкладов признаков');

### Swarmplots(?)

мы также можем воспользоваться swarmplots, которые могут позволить нам более детально оцинить распределение, так как они строятся для каждого отдельного наблюдения.
Если мы закрасим каждую точку масштабированным значением связанного признака, мы можем увидеть как изменяется значение признака вместе с его вкладом.


<i>We can also use swarmplots which allow for a more granular view of the distribution since they plot each individual observation. If we color each point by the associated feature's (scaled) value, we can view how the change in a feature's value changes along with its contribution.</i>

`seaborn` не поддерживает colorbar по умолчанию, так что нам нужно будет добавить его самостоятельно.
Я сделал функию (основываясь на [этой](https://stackoverflow.com/questions/40814612/map-data-points-to-colormap-with-seaborn-swarmplot) stackoverflow answer) Которая дает нам возможность добавить вертикальный colorbar справа от нашего swarmplots.

In [ ]:
from matplotlib.colorbar import ColorbarBase
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
# вспомогательная функция, которая отображает Swarmplot вместе с цветовой шкалой
# использован код, найденный здесь:
# https://stackoverflow.com/questions/40814612/map-data-points-to-colormap-with-seaborn-swarmplot
def swarmplot_with_cbar(cmap, cbar_label, *args, **kwargs):
    fig = plt.gcf()
    ax = sns.swarmplot(*args, **kwargs)
    # удалим легенду, потому что мы хотим показать вместо неё цветную полосу
    ax.legend().remove()
    ## создаем цветовую шкалу ##
    divider = make_axes_locatable(ax)
    ax_cb = divider.new_horizontal(size="3%", pad=0.05)
    fig.add_axes(ax_cb)
    cb = ColorbarBase(ax_cb, cmap=cmap, orientation='vertical')
    cb.set_label(cbar_label, labelpad=10)
    
    return fig

In [ ]:
# мин-макс масштабирование значений объекта позволяет нам использовать цвет
# to indicate high or low feature values
train_scaled_feat_vals = (train_expl_df.groupby('feature')
                                       .value
                                       .transform(lambda x: x/x.max()))

train_expl_df['scaled_feat_vals'] = train_scaled_feat_vals

cmap = plt.get_cmap('viridis')
cbar_label = 'Значение признака %ile'

plt.title('Распределение вкладов признаков (обучающая выборка)')
swarmplot_with_cbar(cmap, cbar_label,  x='feature', y='contribution',
                    hue='scaled_feat_vals', palette='viridis', order=features,
                    data=train_expl_df.loc[train_expl_df.feature!='<BIAS>']);

In [ ]:
test_scaled_feat_vals = (test_expl_df.groupby('feature')
                                      .value
                                      .transform(lambda x: x/x.max()))

test_expl_df['scaled_feat_vals'] = test_scaled_feat_vals

plt.title('Распределение вкладов признаков (тестовая выборка)')
swarmplot_with_cbar(cmap, cbar_label,  x='feature', y='contribution',
                    hue='scaled_feat_vals', palette='viridis', order=features,
                    data=test_expl_df.loc[test_expl_df.feature!='<BIAS>']);

Based on both plots, we can see that faster forty times and higher weights result in more positive contributions.  The other features tend to have most of their contributions hover around 0.  It's also interest to note the gaps in middle of the Wt distributions on both plots.

### Plotting Feature Contributions against Feature Values

Let's plot the feature contributions against the feature values to get a better sense of how they relate to one another.  We can use `seaborn`'s `lmplot` to easily create a grid of these kinds of plots for both our training and testing data.

In [ ]:
fg = sns.lmplot(x='value', y='contribution', col='feature',
                data=train_expl_df.loc[train_expl_df.feature!='<BIAS>'], 
                col_order=features, sharex=False, col_wrap=3, fit_reg=False,
                size=4, scatter_kws={'color':'salmon', 'alpha': 0.5, 's':30})
fg.fig.suptitle('Feature Contributions vs Feature Values (training data)')
fg.fig.subplots_adjust(top=0.90);

In [ ]:
fg = sns.lmplot(x='value', y='contribution', col='feature',
                data=test_expl_df.loc[test_expl_df.feature!='<BIAS>'], 
                col_order=features, sharex=False, col_wrap=3, fit_reg=False, 
                size=4, scatter_kws={'color':'salmon', 'alpha': 0.5, 's':30})
fg.fig.suptitle('Feature Contributions vs Feature Values (testing data)')
fg.fig.subplots_adjust(top=0.90);

In both plots for Wt it's interesting to see the rapid increase in contribution at around 270 lbs.  The model essentially believes that weighing more than 270 is automatically a positive factor for a player, while weighing less than that is a negative one.

One thing to note about these plots is that when we see different contributions (e.g. -0.05, -0.10, -0.15) for the same feature value (e.g. a forty time of 5 seconds) there is probably another feature (or set of features) that is causing these differences. To view such feature interactions we can set the color of the dots to reflect the value of another feature.  Let's take a look at how a player's weight interacts with the contribution of their forty time (at least in the training set).

In [ ]:
# before we actually plot anything we need to do a bit of data manipulation
# let's pivot the data and create a new dataframe where the columns are
# the feature contributions and each row is a player, with the player
# name as the index value
# here are different ways to pivot column values to columns
# https://stackoverflow.com/questions/26255671/pandas-column-values-to-columns
# based on running %%timeit, the groupby method was fastest 
train_contrib_df = (train_expl_df.groupby(['Player','feature'])
                                 .contribution
                                 .aggregate('first')
                                 .unstack())
# add in the feature values
train_feat_contrib_df = train_contrib_df.merge(train_df[['Player'] + features],
                                               how='left', left_index=True, 
                                               right_on='Player',
                                               suffixes=('_contrib', '_value'))
# now we can plot
plt.scatter(x='Forty_value', y='Forty_contrib', c='Wt_value', cmap=cmap,
            data=train_feat_contrib_df)
plt.xlabel('Forty')
plt.ylabel('contribution')
plt.colorbar(label='Wt');

We can see that there is some interaction between weight and forty time. Given a specific forty time, players with higher weights tend to have a more positive (or less negative) contribution.

### Heatmaps
With a little data wrangling and `seaborn`'s `heatmap` function we can take a look at the full set of contributions for each player in our test set.

In [ ]:
def double_heatmap(data1, data2, cbar_label1, cbar_label2,
                   title='', subplot_top=0.86, cmap1='viridis', cmap2='magma', 
                   center1=0.5, center2=0, grid_height_ratios=[1,4],
                   figsize=(14,10)):
    # do the actual plotting
    # here we plot 2 seperate heatmaps one for the predictions and actual percentiles
    # the other for the contributions
    # the reason I chose to do this is because of the difference in magnitudes
    # between the percentiles and the contributions
    fig, (ax,ax2) = plt.subplots(nrows=2, figsize=figsize, 
                                 gridspec_kw={'height_ratios':grid_height_ratios})

    fig.suptitle(title)
    fig.subplots_adjust(hspace=0.02, top=subplot_top)

    # heatmap for actual and predicted percentiles
    sns.heatmap(data1, cmap="viridis", ax=ax, xticklabels=False, center=center1,
                cbar_kws={'location':'top', 
                          'use_gridspec':False, 
                          'pad':0.1,
                          'label': cbar_label1})
    ax.set_xlabel('')

    # heatmap of the feature contributions
    sns.heatmap(data2, ax=ax2, xticklabels=False, center=center2, cmap=cmap2,
                cbar_kws={'location':'bottom', 
                          'use_gridspec':False, 
                          'pad':0.07, 
                          'shrink':0.41,
                          'label': cbar_label2})
    ax2.set_ylabel('');
    return fig

In [ ]:
# get the prediction and actual target values to plot
y_test_and_pred_df = pd.DataFrame(np.column_stack((y_test, y_pred)),
                                  index=test_df.Player,
                                  columns=['true_AV_pctile', 'pred_AV_pctile'])

# let's pivot the data such that the feature contributions are the columns
test_heatmap_df = (test_expl_df.groupby(['Player','feature'])
                               .contribution
                               .aggregate('first')
                               .unstack())

# there may be some NaNs if a feature did not contribute to a prediction, 
# so fill them in with 0s
test_heatmap_df = test_heatmap_df.fillna(0)

# merge our predictions with the the contributions
test_heatmap_df = test_heatmap_df.merge(y_test_and_pred_df, how='left',
                                        right_index=True, left_index=True)
# sort by predictions
test_heatmap_df.sort_values('pred_AV_pctile', ascending=True, inplace=True)

In [ ]:
title = 'Feature contributions to predicted AV %ile \nfor each player in the testing data'
fig = double_heatmap(test_heatmap_df[['true_AV_pctile', 'pred_AV_pctile']].T,
                     test_heatmap_df[features].T, '%ile', 'contribution',
                     title=title)

The plot above consist of two heatmaps. Each column in both heatmaps represents a player. The rows in the top heatmap represent the true and predicted AV percentiles while the rows in the bottom heatmap represent the the feature contributions for each player prediction.

I like the above visualization because it makes it easy to view a large set of predictions and contributions all at once. I definitely prefer it over the alternative of viewing each set of contributions through a printout out of the `DataFrame`.

# Joint Feature Contributions

A benefit of using a tree-ensemble like our random forest model, is that it captures interactions among our features without us explicitly defining them.  However in our attempt to interpret our model we have only looked at the importances and contributions of individual features.  Since we've yet to measure the impact of the feature interactions that the model has found, we have an incomplete picture of what our model is actually doing.  To gain some insight into these feature interactions we can use the `treeinterpeter` package. It uses the same method as before to calculate contributions, but instead of crediting individual features along the decision paths, `treeinterpeter` allows us to credit the feature interactions.

**NOTE:** If you use XGBoost you can use the [xgbfir](https://github.com/limexp/xgbfir) package to inspect feature interactions. 

Let's use `treeinterpreter` on our simple decision tree from before, in order to get an idea of of how joint feature contributions (i.e. the contributions of the feature interactions) are calculated.

In [ ]:
# a reminder of what the tree looks like
Image(graph.create_png())

To get the joint feature contributions, we pass in our estimator and the data to the `predict` function and set `joint_contribution` to `True`.  That should return the prediction, the bias term and the joint feature contributions for our player.

In [ ]:
import treeinterpreter.treeinterpreter as ti
# get the contributions for our simple player example
# who has a 4.6 forty and weighs 260 lbs
# joint_contribution=True gets the joint feature contributions
# when set to False it just returns the individual feature contributions
example_pred, example_bias, example_contrib = ti.predict(tree,
                                                         example.reshape(1, -1),
                                                         joint_contribution=True)

In [ ]:
# same prediction as before
example_pred

In [ ]:
# same bias value as before
example_bias

In [ ]:
example_contrib

The joint contributions are returned as list of dictionaries (in our example one dictionary for our one player), where the keys are numeric tuples representing the features (0 represents Forty and 1 represent Wt) and the values are the joint feature contributions. The joint feature contributions for our simple example are as follows:

$$\underset{\text{AV %ile}}{0.481} = \underset{\text{bias}}{0.414}-\underset{\text{Wt}}{0.097}+\underset{\text{Forty & Wt}}{0.164}$$

The contributions are the same values as before, the difference we see here is that instead of crediting the contribution of 0.164 percentage points to just the player's forty time, we also credit the previous feature, Wt, in the decision path. Remember, we are now crediting feature interactions, not just individual features. We only credit a single feature when it's either at the root node (like Wt is) or if it's the only feature used along a decision path.


Let's get the joint feature contributions for the test set predictions.

In [ ]:
joint_pred, joint_bias, joint_contrib = ti.predict(estimator,
                                                   test_X_imp,
                                                   joint_contribution=True)

In [ ]:
# double check predictions are correct
np.allclose(y_pred, joint_pred)

In [ ]:
# the bias is still the same
joint_bias[:3]

In [ ]:
# 96 observations in test set
len(joint_contrib)

In [ ]:
# tuples representing the column indexes of our features
list(joint_contrib[0].keys())[:3]

In [ ]:
# an example of a joint feature contribution
joint_contrib[0][(0, 1)]

To get the joint feature contributions into more useable format let's match the tuples of indexes to the proper feature names. Then we can construct a `DataFrame` of each player's joint feature contributions, ordered by the absolute value of the contributions.

In [ ]:
def create_ordered_joint_contrib_df(contrib):
    """
    Creates a dataframe from the joint contribution info, where the
    feature combinations are ordered (in descending fashion) by the absolute
    value of the joint contribution.
    """
    df = pd.DataFrame(contrib, columns=['feat_interaction', 'contribution'])
    # get the reordered index    
    new_idx = (df.contribution.abs()
                              .sort_values(inplace=False, ascending=False)
                              .index)
    df = df.reindex(new_idx).reset_index(drop=True)
    return df

# add the names of the feats to the joint contributions
joint_contrib_w_feat_names = []
# for each observation in the join contributions
for obs in joint_contrib:
    # create a list
    obs_contrib = []
    # for each tuple of column indexes
    for k in obs.keys():
        # get the associated feature names
        feature_combo = [features[i] for i in k]
        # get the contribution value
        contrib = obs[k]
        # store that information in the observation individual list
        obs_contrib.append([feature_combo, contrib])
    # append that individual to the large list containing each observations
    # joint feature contributions
    joint_contrib_w_feat_names.append(obs_contrib)

# create an ordered dataframe for each player
joint_contrib_dfs = [create_ordered_joint_contrib_df(contrib)
                     for contrib in joint_contrib_w_feat_names]
# now combine them all
joint_contrib_df = pd.concat(joint_contrib_dfs, keys=test_df.Player, names=['Player'])

# edit feat_interaction column so the values are strings and not lists
joint_contrib_df['feat_interaction'] = joint_contrib_df.feat_interaction.apply(' | '.join) 

In [ ]:
joint_contrib_df.head()

Great, now we have the joint feature contributions for each player in our test set in a nice `DataFrame`.  Let's take a look at how important each feature and feature interaction is to our predictions. To do that we will measure (as a percentage) how much of the total joint contributions an individual feature or feature interaction is responsible for. In other words we are going to measure the relative importance of each feature and feature interaction.

In [ ]:
# first get the sum of the absolute values for each joint feature contribution
abs_imp_joint_contrib = (joint_contrib_df.groupby('feat_interaction')
                                          .contribution
                                          .apply(lambda x: x.abs().sum())
                                           .sort_values(ascending=False))

# then calculate the % of total contribution by dividing by the sum of all absolute vals
rel_imp_join_contrib = abs_imp_joint_contrib / abs_imp_joint_contrib.sum()

rel_imp_join_contrib.head(15)[::-1].plot(kind='barh', color='salmon', 
                                              title='Joint Feature Importances');
plt.ylabel('Features')
plt.xlabel('% of total joint contributions');

It looks like the interaction between a Forty and Wt was the most important feature interaction in our predictions, accounting for over 20% of the joint feature contributions. Overall Forty, Wt and their interaction account for more than 50% of the total feature contributions.

We can also take a look at the distributions of contributions for each feature and feature interaction.

In [ ]:
top_feat_interactions = rel_imp_join_contrib.head(15).index
top_contrib_mask = joint_contrib_df.feat_interaction.isin(top_feat_interactions)
sns.boxplot(y='feat_interaction', x='contribution', 
            data=joint_contrib_df.loc[top_contrib_mask],
            orient='h', order=top_feat_interactions);

And finally let's make another double heatmap plot to observe some of the joint contributions for each prediction in our test set.

In [ ]:
joint_contrib_heatmap_df = (joint_contrib_df[top_contrib_mask]
                               .groupby(['Player','feat_interaction'])
                               .contribution
                               .aggregate('first')
                               .unstack())
joint_contrib_heatmap_df = joint_contrib_heatmap_df.fillna(0)

joint_contrib_heatmap_df = joint_contrib_heatmap_df.merge(y_test_and_pred_df, 
                                                          how='left',
                                                          right_index=True, 
                                                          left_index=True)
# sort by predictions
joint_contrib_heatmap_df.sort_values('pred_AV_pctile', ascending=True, 
                                     inplace=True)

title = 'Top 15 Joint Feature Contributions to predicted AV %ile\n(testing data)'
fig = double_heatmap(joint_contrib_heatmap_df[['true_AV_pctile', 'pred_AV_pctile']].T,
                     joint_contrib_heatmap_df[top_feat_interactions].T, 
                     cbar_label1='%ile', cbar_label2='contribution', 
                     title=title, grid_height_ratios=[1, 7], figsize=(14, 12),
                     subplot_top=0.89)

# PDP and ICE plots

Individual Conditional Expectation (ICE) plots allow us to visualize how changes for a given feature impact the predictions for a set of observations.

Let's use `pycebox` to create an ICE plot to view how changes in the forty yards dash impact our prediction in our training data.

In [ ]:
from pycebox.ice import ice, ice_plot

# pcyebox likes the data to be in a DataFrame so let's create one with our imputed data
# we first need to impute the missing data
train_X_imp_df = pd.DataFrame(train_X_imp, columns=features)

We get the ICE values for our feature of interest (Forty) using the `ice` function.

In [ ]:
forty_ice_df = ice(data=train_X_imp_df, column='Forty', 
                   predict=search.predict)

And then we create the ICE plot with the `ice_plot` function.

In [ ]:
ice_plot(forty_ice_df, c='dimgray', linewidth=0.3)
plt.ylabel('Pred. AV %ile')
plt.xlabel('Forty');

So how was the above plot created and what is it telling us? To create an ICE plot, we first pick a feature of interest. Then for each observation we make predictions across a range of values for that feature, while holding all other features constant. Finally we just visualize those predictions as curves on a plot. By plotting these curves we are able to observe the relationship between the feature of interest and the predicted target variable. 

In our ICE plot above we can see how each player's predicted AV percentile tends to decrease in a non-linear manner between forty times of 4.6 seconds and 5.0 seconds.  We can also see that each player's prediction is impacted in a different manner.  For example, it looks like the player predictions at the top of the plot do not decrease as much as those at the bottom. The differences we see among the curves indicate that there are interactions between the forty times and the other features. 

To inspect feature interactions we can color the ICE curves by another feature.  We can do that by passing in a feature to `ice_plot`'s `color_by` parameter. Let's color each line by the player's weight.

In [ ]:
# new colormap for ICE plot
cmap2 = plt.get_cmap('OrRd')
# set color_by to Wt, in order to color each curve by that player's weight
ice_plot(forty_ice_df, linewidth=0.5, color_by='Wt', cmap=cmap2)
# ice_plot doesn't return a colorbar so we have to add one
# hack to add in colorbar taken from here:
# https://stackoverflow.com/questions/8342549/matplotlib-add-colorbar-to-a-sequence-of-line-plots/11558629#11558629
wt_vals = forty_ice_df.columns.get_level_values('Wt').values
sm = plt.cm.ScalarMappable(cmap=cmap2, 
                           norm=plt.Normalize(vmin=wt_vals.min(), 
                                              vmax=wt_vals.max()))
# need to create fake array for the scalar mappable or else we get an error
sm._A = []
plt.colorbar(sm, label='Wt')
plt.ylabel('Pred. AV %ile')
plt.xlabel('Forty');

From the plot above we can see that heavier players are not impacted in the same manner as lighter ones.

We can also add the PDP by setting the `plot_pdp` to `True` in the `ice_plot` function. To adjust the styling of the PDP line we pass a dictionary of settings to `pdp_kwargs`.

In [ ]:
ice_plot(forty_ice_df, linewidth=.5, color_by='Wt', cmap=cmap2, plot_pdp=True, 
         pdp_kwargs={'c': 'k', 'linewidth': 5})
plt.colorbar(sm, label='Wt')
plt.ylabel('Pred. AV %ile')
plt.xlabel('Forty');

The PDP is the average of all ICE curves on the plot, so the PDP above represents the average change in the predicted AV percentile over the range of forty times.

## Centered ICE Plots

One drawback with our previous ICE plots is that the stacked nature of the lines can make it difficult to observe the differences between the ICE curves. To make it easier to spot those differences we can center or "pinch" the curves at a specific feature value. Typically the minimum is a good centering point. With these centered ICE plots we observe the relative change of the predictions with respect to the predictions at the centered value.

To center our ICE curves at the minimum Forty value we just set `centered` to `True`.

In [ ]:
ice_plot(forty_ice_df, linewidth=.5, color_by='Wt', cmap=cmap2, plot_pdp=True, 
         pdp_kwargs={'c': 'k', 'linewidth': 5}, centered=True)
plt.colorbar(sm, label='Wt')
plt.ylabel('Pred. AV %ile (centered)')
plt.xlabel('Forty');

Note that each line above starts at 0. The y-axis now represents the difference in each player's prediction relative to their prediction at the minimum forty yard dash time  of 4.47 seconds.

Let's take a look at the ICE plots for all our features. To make that easier to do, I created a helper function that can plot out all the ICE plots for each feature. It also adds a rug plot at the bottom of each ICE plot to display information about the distribution of the data.

In [ ]:
def plot_ice_grid(dict_of_ice_dfs, data_df, features, ax_ylabel='', nrows=3, 
                  ncols=3, figsize=(12, 12), sharex=False, sharey=True, 
                  subplots_kws={}, rug_kws={'color':'k'}, **ice_plot_kws):
    """A function that plots ICE plots for different features in a grid."""
    fig, axes = plt.subplots(nrows=nrows, 
                             ncols=ncols, 
                             figsize=figsize,
                             sharex=sharex,
                             sharey=sharey,
                             **subplots_kws)
    # for each feature plot the ice curves and add a rug at the bottom of the 
    # subplot
    for f, ax in zip(features, axes.flatten()):
        ice_plot(dict_of_ice_dfs[f], ax=ax, **ice_plot_kws)
        # add the rug
        sns.distplot(data_df[f], ax=ax, hist=False, kde=False, 
                     rug=True, rug_kws=rug_kws)
        ax.set_title('feature = ' + f)
        ax.set_ylabel(ax_ylabel)
        sns.despine()
        
    # get rid of blank plots
    for i in range(len(features), nrows*ncols):
        axes.flatten()[i].axis('off')

    return fig

In [ ]:
# create dict of ICE data for grid of ICE plots
train_ice_dfs = {feat: ice(data=train_X_imp_df, column=feat, predict=estimator.predict) 
                 for feat in features}

fig = plot_ice_grid(train_ice_dfs, train_X_imp_df, features,
                    ax_ylabel='Pred. AV %ile', alpha=0.3, plot_pdp=True,
                    pdp_kwargs={'c': 'red', 'linewidth': 3},
                    linewidth=0.5, c='dimgray')
fig.tight_layout()
fig.suptitle('ICE plots (training data)')
fig.subplots_adjust(top=0.89);

In [ ]:
fig = plot_ice_grid(train_ice_dfs, train_X_imp_df, features, 
                    ax_ylabel='Pred AV %ile (centered)',
                    alpha=.2, plot_points=False, plot_pdp=True,
                    pdp_kwargs={"c": "red", "linewidth": 3},
                    linewidth=0.5, c='dimgray', centered=True,
                    sharey=False, nrows=4, ncols=2, figsize=(11,16))
fig.tight_layout()
fig.suptitle('Centered ICE plots (training data)')
fig.subplots_adjust(top=0.9)

## Two-dimensional PDPs
We can also create plot the PDPs for 2 features at once. This allows us to better understand interactions between the features and how they impact the predictions. We'll use the the `pdpbox` library to create 2-D PDPs.

**NOTE** `pdpbox` can also create the same kind of ice plots that we created with `pycebox` above. However `pycebox` doesn't have support for 2-D PDPs.

In [ ]:
import itertools
from pdpbox import pdp

I'm just going to jump into creating a grid of 2-D PDPs and explain whats happening in comments of the code below.

In [ ]:
from matplotlib.text import Text

def plot_2d_pdp_grid(pdp_inters, feature_pairs,
                     ncols=3, nrows=4, figsize=(13, 16),
                     xaxis_font_size=12, yaxis_font_size=12,
                     contour_line_fontsize=12,
                     tick_labelsize=10, x_quantile=None, 
                     plot_params=None, subplots_kws={}):
    """Plots a grid of 2D PDP plots."""
    # create our subplots to plot our PDPs on
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, 
                             figsize=figsize, **subplots_kws)

    # for each feature pair, plot the 2-D pdp
    for pdp_inter, feat_pair, ax in zip(pdp_inters, feature_pairs, axes.flatten()):
    
        # use pdpbox's _pdp_contour_plot function to actually plot the 2D pdp
        pdp._pdp_contour_plot(pdp_inter, feat_pair, 
                              x_quantile=x_quantile, ax=ax, 
                              plot_params=plot_params,
                              fig=None)
        # adjust some font sizes
        ax.tick_params(labelsize=tick_labelsize)
        ax.xaxis.get_label().set_fontsize(xaxis_font_size)
        ax.yaxis.get_label().set_fontsize(yaxis_font_size)
    
        # set the contour line fontsize
        for child in ax.get_children():
            if isinstance(child, Text):
                child.set(fontsize=contour_line_fontsize)   
    
    # get rid of empty subplots
    for i in range(len(pdp_inters), nrows*ncols):
        axes.flatten()[i].axis('off')
        
    return fig

In [ ]:
# get each possible feature pair combination
feature_pairs = [list(feat_pair) for feat_pair in itertools.combinations(features, 2)]
# we will only plot the feature iteractions that invlove either Forty or Wt
# just to avoid making soooo many plots
forty_wt_feat_pairs = [fp for fp in feature_pairs if 'Forty' in fp or 'Wt' in fp]
# now calculate the data for the pdp interactions
# we can do that with pdpbox's pdp_interact function
# in the current development version on github, parallelization is supported
# but it didn't work for me so I resorted to using that multiprocess helper
# function from before
train_feat_inters = multiproc_iter_func(N_JOBS, forty_wt_feat_pairs, 
                                        pdp.pdp_interact, 'features',
                                        model=estimator, train_X=train_X_imp_df)

In [ ]:
# and now plot a grid of PDP interaction plots
# NOTE that the contour colors do not represent the same values
# across the different subplots
fig = plot_2d_pdp_grid(train_feat_inters, forty_wt_feat_pairs)
fig.tight_layout()
fig.suptitle('PDP Interaction Plots (training data)', fontsize=20)
fig.subplots_adjust(top=0.95);

# LIME

Local interpretable model-agnostic explanations (LIME) allow us to explain individual predictions for "black box" models by creating local, interpretable, surrogate models. We fit a local model using the following recipe (which I copied from [Christop Molnar's](https://twitter.com/ChristophMolnar) great book, [Interpretable Machine Learning](https://christophm.github.io/interpretable-ml-book/lime.html)):

> 
- Choose your instance of interest for which you want to have an explanation of its black box prediction.
- Perturb your dataset and get the black box predictions for these new points.
- Weight the new samples by their proximity to the instance of interest.
- Fit a weighted, interpretable model on the dataset with the variations.
- Explain prediction by interpreting the local model.

There are different packages that implement LIME (including `eli5` and another package I just discovered called `Skater`). Here we will use the original `lime` package created by the authors of the LIME [paper](https://arxiv.org/abs/1602.04938).

In [ ]:
import lime
from lime.lime_tabular import LimeTabularExplainer

Let's create our LIME explainer and explain an instance from our test set.

In [ ]:
# create the explainer by passing our training data, 
# setting the correct modeling mode, pass in feature names and
# make sure we don't discretize the continuous features
explainer = LimeTabularExplainer(train_X_imp_df, mode='regression', 
                                 feature_names=features, 
                                 random_state=RANDOM_STATE, 
                                 discretize_continuous=False) 

In [ ]:
test_X_imp_df = pd.DataFrame(test_X_imp, columns=features)
# the number of features to include in our predictions
num_features = len(features)
# the index of the instance we want to explaine
exp_idx = 2
exp = explainer.explain_instance(test_X_imp_df.iloc[exp_idx,:].values, 
                                 estimator.predict, num_features=num_features)

Cool, now we have our explanation, let's inspect it.

In [ ]:
# the prediction made by the local surrogate model
exp.local_pred[0]

In [ ]:
# the bias term for the local explanation
exp.intercept[0]

In [ ]:
# a plot of the weights for each feature
exp.as_pyplot_figure();

In [ ]:
# # and a prettier output that we can view in the notebook
# # it looks like it messes with the blog post so I've commented it out
# exp.show_in_notebook()

Great, we have an explanation for that one instance, what about the rest of the test set? We can use the `apply` method to get explanation for the whole test set.

**NOTE:** We can't parallelize `explain_instance` with the multiprocessing function since `explain_instance` is a bound method.

In [ ]:
lime_expl = test_X_imp_df.apply(explainer.explain_instance, 
                                predict_fn=estimator.predict, 
                                num_features=num_features,
                                axis=1)

Before moving on we should double check and see that the local predictions from our surrogate models match our actual predictions. We can judge the local prediction by looking at either the root-mean-squared error or the R<sup>2</sup>

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# get all the lime predictions
lime_pred = lime_expl.apply(lambda x: x.local_pred[0])
# RMSE of lime pred
mean_squared_error(y_pred, lime_pred)**0.5

In [ ]:
# r^2 of lime predictions
r2_score(y_pred, lime_pred)

If you aren't satisfied with the fit of the local surrogate models you can try to improve them by playing around with the `kernel_width` parameter in `LimeTabularExplainer`. We will try and improve the surrogate models by decreasing the kernel width to make the fits more local.

In [ ]:
# new explainer with smaller kernel_width
better_explainer = LimeTabularExplainer(train_X_imp_df, mode='regression', 
                                        feature_names=features, 
                                        random_state=RANDOM_STATE, 
                                        discretize_continuous=False,
                                        kernel_width=1) 

In [ ]:
better_lime_expl = test_X_imp_df.apply(better_explainer.explain_instance, 
                                       predict_fn=estimator.predict, 
                                       num_features=num_features,
                                       axis=1)

In [ ]:
# get all the lime predictions
better_lime_pred = better_lime_expl.apply(lambda x: x.local_pred[0])
# RMSE of lime pred
mean_squared_error(y_pred, better_lime_pred)**0.5

In [ ]:
# r^2 of lime predictions
r2_score(y_pred, better_lime_pred)

Our new local predictions better match our actual predictions. To view all of our explanations at once we can create heatmap in the same manner we did when looking at the feature contributions. To do that we need to create a `DataFrame` with each instance's feature weights and bias term from the LIME explanation.

In [ ]:
# construct a DataFrame with all the feature weights and bias terms from LIME
# create an individual dataframe for each explanation
lime_dfs = [pd.DataFrame(dict(expl.as_list() + [('bias', expl.intercept[0])]), index=[0]) 
            for expl in better_lime_expl]
# then concatenate them into one big DataFrame
lime_expl_df = pd.concat(lime_dfs, ignore_index=True)

lime_expl_df.head()

Now that we have the weights for each feature we can measure their prediction contributions by multiplying the weights by the actual feature values. But before we do that we need to scale the data in our test set since LIME scales the data inside the explainer when the data is not discretized.

In [ ]:
# scale the data
scaled_X = (test_X_imp_df - explainer.scaler.mean_) / explainer.scaler.scale_
# calc the lime feature contributions
lime_feat_contrib = lime_expl_df[features] * scaled_X

# add on bias term, actual av %ile and predicted %ile
other_lime_cols = ['bias', 'true_AV_pctile', 'pred_AV_pctile']
lime_feat_contrib[other_lime_cols] = pd.DataFrame(np.column_stack((lime_expl_df.bias,
                                                                   y_test_and_pred_df)))

lime_feat_contrib.sort_values('pred_AV_pctile', inplace=True)

lime_feat_contrib.head()

Now to plot each set of explanations using our `double_heatmap` function. Unlike previous heatmaps, we will include the bias terms since the surrogate models that LIME creates can have different bias terms for each player.

In [ ]:
title = 'LIME Feature Contributions for each prediction in the testing data'
fig = double_heatmap(lime_feat_contrib[['true_AV_pctile', 'pred_AV_pctile']].T,
                     lime_feat_contrib.loc[:, :'bias'].T, title=title,
                     cbar_label1='%ile', cbar_label2='contribution', 
                     subplot_top=0.9)
# set the x-axis label for the bottom heatmap
# fig has 4 axes object, the first 2 are the heatmaps, the other 2 are the colorbars
fig.axes[1].set_xlabel('Player');

# SHAP

SHAP (SHapley Additive exPlanations) is a recent method for model interpretation that leverages game theory to help measure the impact of the features on the predictions. What’s the benefit of using the SHAP method for individual feature contributions over the decision path method from before? Well with the decision path method we have to traverse down the decision tree crediting each feature for the difference in the predictions. This can result in individual contributions that favor features found in splits lower in the tree. The SHAP method doesn't have that problem as it doesn't rely on the order of the features specified by the tree, instead it calculates the contributions by basically averaging the differences in predictions over every possible feature ordering.  For more on this topic I suggest reading Scott Lindburg’s (one of the authors of the SHAP paper) [blog post](https://towardsdatascience.com/interpretable-machine-learning-with-xgboost-9ec80d148d27).  And for a good explanation of how SHAP values are calculated I suggest reading the [chapter on SHAP](https://christophm.github.io/interpretable-ml-book/shapley.html#) from Christopher Molnar's book.


Now let's actually use the SHAP method to explain our predictions.

In [ ]:
import shap

# create our SHAP explainer
shap_explainer = shap.TreeExplainer(estimator)
# calculate the shapley values for our test set
test_shap_vals = shap_explainer.shap_values(test_X_imp)

The `shap` package provides us with convenience functions to help us plot the SHAP values for our predictions. We can use `force_plot` to inspect individual predictions.

In [ ]:
# load JS in order to use some of the plotting functions from the shap
# package in the notebook
shap.initjs()

In [ ]:
# plot the explanation for a single prediction
shap.force_plot(test_shap_vals[0, :], test_X_imp_df.iloc[0, :])

In the above explanation we can see that there is a base value (i.e. a bias term) of 0.4219, and that each feature pushes (i.e. adds to) that value in order to reach a final prediction of 0.46.
  

We can also use the `force_plot` function to look at the explanations for our whole dataset. When using the function in a notebook, it produces an interactive plot that allows you to inspect the SHAP values for each observation by hovering the mouse over the plot. By default the observations are clustered together by how similar they are. For example, the first 17 observations in the plot below are players whose weights have a very large positive impact on their predictions. 

In [ ]:
shap.force_plot(test_shap_vals, test_X_imp_df)

To plot the distribution of each feature's SHAP we can use the `summary_plot` function.

In [ ]:
shap.summary_plot(test_shap_vals, test_X_imp_df, auto_size_plot=False)

With the `dependence_plot` function we can see how a feature's SHAP values change over the range of feature values. The function automatically colors each point on the plot by a 2nd feature, allowing us to better understand the interaction effects. 

In [ ]:
for feat in features:
    shap.dependence_plot(feat, test_shap_vals, test_X_imp_df, 
                         dot_size=100)

And of course a heatmap of the SHAP values.

In [ ]:
test_shap_df = pd.DataFrame(np.column_stack((test_shap_vals, y_test_and_pred_df)),
                            columns= features + ['bias', 'true_AV_pctile', 
                                                 'pred_AV_pctile'])
test_shap_df.sort_values('pred_AV_pctile', inplace=True)

title = 'SHAP Values for each prediction in the testing data'
fig = double_heatmap(test_shap_df[['true_AV_pctile', 'pred_AV_pctile']].T,
                     test_shap_df[features].T, '%ile', 'SHAP Value',
                     title=title, subplot_top=0.89)
fig.axes[1].set_xlabel('Player');

Hopefully you found this blog post helpful.  If you see any mistakes, have any questions or suggestions  [or if you're hiring :)] you can email me at savvas.tjortjoglou@gmail.com, hit me up on Twitter [@savvastj](https://twitter.com/savvastj), or just leave a comment below.

If you like this post and want to support my blog you can check out my patreon page [here](https://www.patreon.com/savvastj).

# Resources

Here are a list of resources that I found helpful when writing up this post:

**General**
- [Interpretable Machine Learning](https://christophm.github.io/interpretable-ml-book/)

**Feature Importance**
- [How are feature_importances in RandomForestClassifier determined?](https://stackoverflow.com/questions/15810339/how-are-feature-importances-in-randomforestclassifier-determined)
- [Selecting good features – Part III: random forests](http://blog.datadive.net/selecting-good-features-part-iii-random-forests/)

**Feature Contributions**
- [Interpreting random forests](http://blog.datadive.net/interpreting-random-forests/)
- [Random forest interpretation with scikit-learn](http://blog.datadive.net/random-forest-interpretation-with-scikit-learn/)
- [Random forest interpretation – conditional feature contributions](http://blog.datadive.net/random-forest-interpretation-conditional-feature-contributions/)

**ICE plots and PDPs**
- [Peeking Inside the Black Box: Visualizing Statistical Learning with Plots of Individual Conditional Expectation](https://arxiv.org/abs/1309.6392)

**LIME**
- ["Why Should I Trust You?": Explaining the Predictions of Any Classifier](https://arxiv.org/abs/1602.04938)
- [Why your relationship is likely to last (or not): using Local Interpretable Model-Agnostic Explanations (LIME)](http://blog.fastforwardlabs.com/2017/09/01/LIME-for-couples.html)
- [Understanding LIME](https://cran.r-project.org/web/packages/lime/vignettes/Understanding_lime.html)

**SHAP**
- [Interpretable Machine Learning with XGBoost](https://towardsdatascience.com/interpretable-machine-learning-with-xgboost-9ec80d148d27)
- [Consistent Individualized Feature Attribution for Tree Ensembles](https://arxiv.org/abs/1802.03888)

**NFL Combine/Draft**
- The NFL Combine Actually Matters
    - [Part 1](http://harvardsportsanalysis.org/2015/02/the-nfl-combine-actually-matters/)
    - [Part 2](http://harvardsportsanalysis.org/2015/02/the-combine-actually-matters-part-2/)
    - [Part 3](http://harvardsportsanalysis.org/2015/02/the-combine-actually-matters-part-3-predicting-the-draft/)

    
**Videos**
- [Machine Learning and Interpretability](https://www.youtube.com/watch?v=NxYCY8-Qfx0)
- [Towards interpretable reliable models](https://www.youtube.com/watch?v=B3PtcF-6Dtc)
- [Interpretable Machine Learning Using LIME Framework](https://www.youtube.com/watch?v=CY3t11vuuOM)
- [Explaining behavior of Machine Learning models with eli5 library](https://www.youtube.com/watch?v=s-yT5Is1G1A)

**Model Interpretability Packages**
- [treeinterpreter](https://github.com/andosa/treeinterpreter)
- [eli5](https://github.com/TeamHG-Memex/eli5/tree/master/eli5)
- [pycebox](https://github.com/AustinRochford/PyCEbox)
- [pdpbox](https://github.com/SauceCat/PDPbox)
- [lime](https://github.com/marcotcr/lime)
- [shap](https://github.com/slundberg/shap)
- [Skater](https://github.com/datascienceinc/Skater)
    
As always you can find the notebook and data used for this post on [github](https://github.com/savvastj/model_interpretability_post).